#Eksplorasi Data

In [ ]:
import cv2
import os

video_path = '0X156A170A2372A74D.avi'
output_folder = 'frames74D'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = total_frames / fps if fps > 0 else 0

print(f"Path video     : {video_path}")
print(f"Resolusi       : {width} x {height}")
print(f"FPS            : {fps}")
print(f"Jumlah frame   : {total_frames}")
print(f"Durasi (detik) : {duration:.2f} detik")

#Ekstraksi frame dari video

In [ ]:
import cv2
import os

video_path = '0X156A170A2372A74D.avi'

output_folder = 'frames74D'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

frame_count = 0
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

    frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
print(f"Total frame berhasil diambil: {len(frames)}")

In [ ]:
import matplotlib.pyplot as plt
import cv2
import math

total_frames = len(frames)

cols = math.ceil(math.sqrt(total_frames))
rows = math.ceil(total_frames / cols)

plt.figure(figsize=(cols * 2, rows * 2))

for i in range(total_frames):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB))
    plt.title(f'{i}', fontsize=6)
    plt.axis('off')

plt.tight_layout()
plt.show()

#Pre-processing

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

input_folder = 'frames74D'
output_folder = 'processed74D_frames'
os.makedirs(output_folder, exist_ok=True)

sharpening_kernel = np.array([[0, -1, 0],
                              [-1, 5, -1],
                              [0, -1, 0]])

clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))

example_file = None

for filename in sorted(os.listdir(input_folder)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
        path = os.path.join(input_folder, filename)
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        filtered = cv2.medianBlur(image, 5)
        enhanced = clahe.apply(filtered)
        blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
        sharpened = cv2.filter2D(blurred, -1, sharpening_kernel)
        adaptive_thresh = cv2.adaptiveThreshold(
            sharpened, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            11, -2
        )
        kernel = np.ones((3, 3), np.uint8)
        morph = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel, iterations=1)
        morph = cv2.dilate(morph, kernel, iterations=1)
        contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask = np.zeros_like(morph)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 1000:
                cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)
        resized = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_AREA)
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, resized)
        if example_file is None:
            example_file = (image, filtered, enhanced, sharpened, adaptive_thresh, morph, mask, resized)

if example_file:
    titles = ['Original', 'Median Filter', 'CLAHE', 'Sharpened', 'Threshold', 'Morphology', 'Masking Final', 'Resized']
    images = list(example_file)

    plt.figure(figsize=(28, 6))
    for i in range(len(images)):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.title(titles[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("Tidak ditemukan gambar di folder input.")


#Thresholding dan Deteksi Gerakan dengan Frame Differencing

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

processed_folder = 'processed74D_frames'
output_motion_folder = 'diff_output74D'
os.makedirs(output_motion_folder, exist_ok=True)

frame_files = sorted([f for f in os.listdir(processed_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

if len(frame_files) < 2:
    print("Butuh minimal dua frame untuk deteksi gerakan.")
else:
    for i in range(len(frame_files) - 1):
        path1 = os.path.join(processed_folder, frame_files[i])
        path2 = os.path.join(processed_folder, frame_files[i + 1])

        frame1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
        frame2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

        diff = cv2.absdiff(frame1, frame2)

        _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

        output_filename = f"motion_{i:04d}.png"
        output_path = os.path.join(output_motion_folder, output_filename)
        cv2.imwrite(output_path, thresh)

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(frame1, cmap='gray')
        plt.title(f'Frame {i}')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(frame2, cmap='gray')
        plt.title(f'Frame {i+1}')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(thresh, cmap='gray')
        plt.title('Motion (Thresholded)')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    print(f"Hasil deteksi gerakan disimpan di folder: {output_motion_folder}")


#Perbaikan Hasil Segmentasi dan Operasi Morfologi

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

processed_folder = 'processed74D_frames'
output_morph_folder = 'morph_output74D'
os.makedirs(output_morph_folder, exist_ok=True)

frame_files = sorted([f for f in os.listdir(processed_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

for filename in frame_files:
    path = os.path.join(processed_folder, filename)

    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    _, thresh_otsu = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    opened = cv2.morphologyEx(thresh_otsu, cv2.MORPH_OPEN, kernel)

    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)

    output_path = os.path.join(output_morph_folder, f"morph_{filename}")
    cv2.imwrite(output_path, closed)

    plt.figure(figsize=(16, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(img, cmap='gray')
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(thresh_otsu, cmap='gray')
    plt.title('Otsu Thresholding')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(opened, cmap='gray')
    plt.title('After Opening')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.imshow(closed, cmap='gray')
    plt.title('After Closing')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

print(f"Hasil segmentasi dan morfologi disimpan di folder: {output_morph_folder}")

#Ekstraksi dan Seleksi Kontur

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

input_folder = 'morph_output74D'
output_folder = 'kontur_output'
os.makedirs(output_folder, exist_ok=True)

image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    contours, _ = cv2.findContours(image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    output_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_path, contour_image)

    plt.figure(figsize=(5, 5))
    plt.title(f'Kontur: {image_file}')
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Path folder
kontur_folder = 'kontur_output'
original_folder = 'processed74D_frames'
output_folder = 'mask_output'
os.makedirs(output_folder, exist_ok=True)

image_files = [f for f in os.listdir(kontur_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    base_filename = image_file.replace('morph_', '').replace('kontur_', '')
    kontur_path = os.path.join(kontur_folder, image_file)
    original_path = os.path.join(original_folder, base_filename)

    kontur_img = cv2.imread(kontur_path, cv2.IMREAD_GRAYSCALE)
    original_img = cv2.imread(original_path)

    if kontur_img is None:
        print(f"Gagal membaca file kontur: {kontur_path}")
        continue
    if original_img is None:
        print(f"File tidak ditemukan: {original_path}")
        continue

    contours, _ = cv2.findContours(kontur_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    image_with_contours = original_img.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)

    output_path = os.path.join(output_folder, base_filename)
    cv2.imwrite(output_path, image_with_contours)

    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(image_with_contours, cv2.COLOR_BGR2RGB))
    plt.title(f'Kontur: {base_filename}')
    plt.axis('off')
    plt.show()

print(f"Hasil kontur dengan latar asli disimpan di folder: {output_folder}")


In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

kontur_folder = 'kontur_output'
original_folder = 'processed74D_frames'
mask_output_folder = 'mask_output2'
os.makedirs(mask_output_folder, exist_ok=True)

image_files = [f for f in os.listdir(kontur_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    base_filename = image_file.replace('morph_', '').replace('kontur_', '')

    kontur_path = os.path.join(kontur_folder, image_file)
    original_path = os.path.join(original_folder, base_filename)

    kontur_img = cv2.imread(kontur_path, cv2.IMREAD_GRAYSCALE)
    original_img = cv2.imread(original_path)

    if original_img is None or kontur_img is None:
        print(f"File tidak ditemukan: {image_file}")
        continue

    contours, _ = cv2.findContours(kontur_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    masked_img = original_img.copy()

    cv2.drawContours(masked_img, contours, -1, (0, 255, 0), thickness=cv2.FILLED)

    overlay = cv2.addWeighted(original_img, 0.6, masked_img, 0.4, 0)

    output_path = os.path.join(mask_output_folder, base_filename)
    cv2.imwrite(output_path, overlay)

    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title(f'Masked Kontur: {base_filename}')
    plt.axis('off')
    plt.show()

print(f"Hasil mask warna disimpan di folder: {mask_output_folder}")

#Menjadikan hasil sebagai video lagi

In [ ]:
import cv2
import os

input_folder = 'mask_output2'
output_video = 'hasil_segmentasi22.avi'

frame_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg')])

first_frame = cv2.imread(os.path.join(input_folder, frame_files[0]))
height, width, _ = first_frame.shape

out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'XVID'), 50.0, (width, height))

for frame_name in frame_files:
    frame_path = os.path.join(input_folder, frame_name)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Gagal baca frame: {frame_name}")
        continue

    out.write(frame)

out.release()
print(f" Video berhasil dibuat dan disimpan sebagai: {output_video}")
